**Этот ноутбук — упражнение курса [Intro to Deep Learning](https://www.kaggle.com/learn/intro-to-deep-learning).  Вы можете обратиться к уроку по [этой ссылке](https://www.kaggle.com/ryanholbrook/overfitting-and-underfitting).**

---



# Введение #

В этом упражнении вы научитесь улучшать результаты обучения, добавляя callback ранней остановки, чтобы предотвратить переобучение.

Когда будете готовы, запустите следующую ячейку, чтобы всё подготовить!


In [ ]:
# Setup plotting
import matplotlib.pyplot as plt

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

Сначала загрузим набор данных *Spotify*. Ваша задача — предсказать популярность песни по разным аудиопризнакам, таким как `'tempo'`, `'danceability'` и `'mode'`.


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

spotify = pd.read_csv('../dataset/spotify.csv')

X = spotify.copy().dropna()
y = X.pop('track_popularity')
artists = X['track_artist']

features_num = ['danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms']
features_cat = ['playlist_genre']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

# We'll do a "grouped" split to keep all of an artist's songs in one
# split or the other. This is to help prevent signal leakage.
def group_split(X, y, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

X_train, X_valid, y_train, y_valid = group_split(X, y, artists)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100 # popularity is on a scale 0-100, so this rescales to 0-1.
y_valid = y_valid / 100

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

Начнём с самой простой сети — линейной модели. У этой модели низкая емкость.

Запустите следующую ячейку без изменений, чтобы обучить линейную модель на наборе данных *Spotify*.


In [ ]:
model = keras.Sequential([
    layers.Dense(1, input_shape=input_shape),
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
    verbose=0, # suppress output since we'll plot the curves
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

Нередко кривые выглядят как «клюшка», как на графике здесь. Из‑за этого финальную часть обучения трудно рассмотреть, поэтому начнём с 10‑й эпохи:


In [ ]:
# Start the plot at epoch 10
history_df.loc[10:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

# 1) Оцените базовую модель

Как вы думаете? Эта модель недообучается, переобучается или всё в порядке?


Теперь добавим емкость нашей сети. Добавим три скрытых слоя по 128 нейронов каждый. Запустите следующую ячейку, чтобы обучить сеть и посмотреть кривые обучения.


In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

# 2) Добавьте емкость

Как вы оцениваете эти кривые? Недообучение, переобучение или всё в порядке?


# 3) Определите callback ранней остановки

Теперь определите callback ранней остановки, который ждёт 5 эпох (`patience`) улучшения валидационных потерь хотя бы на `0.001` (`min_delta`) и сохраняет веса с лучшей потерей (`restore_best_weights`).


In [ ]:
from tensorflow.keras import callbacks

# YOUR CODE HERE: define an early stopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

Теперь запустите эту ячейку, чтобы обучить модель и получить кривые обучения. Обратите внимание на аргумент `callbacks` в `model.fit`.


In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),    
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
    callbacks=[early_stopping]
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

# 4) Обучите и интерпретируйте

Это улучшение по сравнению с обучением без ранней остановки?


Если хотите, поэкспериментируйте с `patience` и `min_delta`, чтобы увидеть, какое это даст различие.

# Дальше #

Переходите к [**уроку о паре специальных слоёв**](https://www.kaggle.com/ryanholbrook/dropout-and-batch-normalization): batch normalization и dropout.


---




*Есть вопросы или комментарии? Загляните на [форум обсуждения курса](https://www.kaggle.com/learn/intro-to-deep-learning/discussion), чтобы пообщаться с другими участниками.*
